In [1]:

from sklearn.model_selection import train_test_split
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import pandas as pd
import numpy as np
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Activation, Dropout, Flatten, Dense
import tensorflow as tf
from os import listdir
from PIL import Image
import glob, os

from os.path import isfile, join
import numpy as np
from pandas import DataFrame
import pandas as pd
from matplotlib import pyplot as plt
#from __future__ import print_function

from SRC.transform import *
#estas son mis funciones traidas del jupyter contiguo

In [2]:
paths={"black":"INPUT/black/",
      "white":"INPUT/white/",
      "green":"INPUT/green/",
      "red":"INPUT/red/"}
images={}
for color,path in paths.items():
    images[color]=[]
    for im in listdir(path):
        if im.endswith("jpg"):
            #size=(64,64)
            a=resize_image(path+im,(64,64))
            images[color].append(np.asarray(a))
    images[color]=np.array(images[color])

In [3]:
X= np.concatenate(list(images.values()),axis=0)
y = np.concatenate([[k]* v.shape[0] for k,v in images.items()])

# Division entre la x y la y para test
### Ojo, normalmente convertiriamos el onehotencoder antes de esta division para ahorrarnos un paso

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [5]:
img_width, img_height = 64, 64
img_rows, img_cols = 64, 64

# Prepare data to feed the NN
num_classes = 4

# Ask keras which format to use depending on used backend and arrange data as expected
if K.image_data_format() == 'channels_first':
    X_train = x_train.reshape(X_train.shape[0], 3, img_rows, img_cols)
    X_test = x_test.reshape(X_test.shape[0], 3, img_rows, img_cols)
    input_shape = (3, img_width, img_height)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 3)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 3)
    input_shape = (img_width, img_height, 3)

# Incoming data is in uint8. Cast the input data images to be floats in range [0.0-1.0]  
X_train = tf.keras.utils.normalize(X_train,axis=1)
X_test = tf.keras.utils.normalize(X_test,axis=1)
#X_train = X_train.astype('float32') / 255
#X_test = X_test.astype('float32') / 255

print('x_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

x_train shape: (8388, 64, 64, 3)
8388 train samples
2097 test samples


In [6]:
#red neuronal propiamente dicha

In [7]:
img_width, img_height = 64, 64

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(tf.keras.layers.Flatten())

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(4))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

### Proceso de cambio de las y para poder utilizarlas en el modelo. 
### Las necesitamos en el formato de array sin strings y getdummies no nos sirvio ya que 
### no podiamos obtener el color con posterioridad.

In [8]:
# importo aqui para que quede reflejado el modulo que utilizamos
from sklearn.preprocessing import OneHotEncoder

In [9]:
y_test


array(['white', 'white', 'white', ..., 'white', 'black', 'red'],
      dtype='<U5')

In [10]:
# Hacemos el reshape, luego onehot encoder, luego fit y por ultimo toarray
y_test= y_test.reshape(-1, 1)
y_test

array([['white'],
       ['white'],
       ['white'],
       ...,
       ['white'],
       ['black'],
       ['red']], dtype='<U5')

In [11]:
##3
enc = OneHotEncoder()
enc.fit(y_test)  

OneHotEncoder()

In [12]:
y_test = enc.transform(y_test).toarray()

In [13]:
#OJO
enc.get_feature_names()

array(['x0_black', 'x0_green', 'x0_red', 'x0_white'], dtype=object)

In [14]:
#por fin y_test esta en la forma que queremos
y_test


array([[0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       ...,
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.]])

In [15]:
#ahora hago lo mismo con y Y_train

In [16]:
y_train

array(['white', 'black', 'black', ..., 'black', 'white', 'white'],
      dtype='<U5')

In [17]:
y_train= y_train.reshape(-1, 1)

In [18]:
enc = OneHotEncoder()
enc.fit(y_train)  

OneHotEncoder()

In [19]:
y_train = enc.transform(y_train).toarray()

In [20]:
y_train

array([[0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       ...,
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.]])

In [21]:
#ha funcionado. Tenemos y_test e y_train en el formato que necesitabamos
# podiamos haberlo hecho antes de dividir el split del train pero he decidido hacerlo dos veces para practicar

In [22]:
#OJO
enc.get_feature_names()

array(['x0_black', 'x0_green', 'x0_red', 'x0_white'], dtype=object)

In [23]:
enc.inverse_transform([[0,1,0,0]])

array([['green']], dtype='<U5')

In [24]:
enc.get_params()

{'categories': 'auto',
 'drop': None,
 'dtype': numpy.float64,
 'handle_unknown': 'error',
 'sparse': True}

In [25]:
# Codigo para para correr el modelo de red neuronal. 

In [26]:
batch_size = 100
epochs = 20

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test))



Epoch 1/20
84/84 [==============================] - 12s 141ms/step - loss: 0.3481 - accuracy: 0.7067 - val_loss: 0.2071 - val_accuracy: 0.8355
Epoch 2/20
84/84 [==============================] - 13s 155ms/step - loss: 0.1953 - accuracy: 0.8615 - val_loss: 0.1349 - val_accuracy: 0.8879
Epoch 3/20
84/84 [==============================] - 14s 173ms/step - loss: 0.1315 - accuracy: 0.9105 - val_loss: 0.0948 - val_accuracy: 0.9180
Epoch 4/20
84/84 [==============================] - 14s 170ms/step - loss: 0.1003 - accuracy: 0.9311 - val_loss: 0.0737 - val_accuracy: 0.9423
Epoch 5/20
84/84 [==============================] - 14s 171ms/step - loss: 0.0845 - accuracy: 0.9412 - val_loss: 0.0607 - val_accuracy: 0.9642
Epoch 6/20
84/84 [==============================] - 14s 170ms/step - loss: 0.0721 - accuracy: 0.9502 - val_loss: 0.0557 - val_accuracy: 0.9619
Epoch 7/20
84/84 [==============================] - 14s 170ms/step - loss: 0.0665 - accuracy: 0.9561 - val_loss: 0.0573 - val_accuracy: 0.9604

In [27]:
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense

In [28]:
model.save("model.cars_colours")
print("Saved model to disk")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: model.cars_colours/assets
Saved model to disk


In [ ]:
# PRUEBAS

In [ ]:
X_test[45].shape
model.predict(np.array([X_test[2000]]))
#nos dice cual es la prediccion del modelo

In [ ]:
enc.get_feature_names()
# y hay que recordar que los colores marcados son:

In [ ]:
enc.inverse_transform([[9.9999976e-01, 6.4836332e-09, 1.1157968e-09, 1.2618054e-12]])
#nos da el resultado mas probable en una tuple. 


In [ ]:
X_test[2000].shape

In [ ]:
model.predict(np.array([X_test[2000]]))

In [ ]:
enc.inverse_transform([[9.9999976e-01, 6.4836332e-09, 1.1157968e-09, 1.2618054e-12]])[0][0]
# le digo que me muestre solo el primer elemento, que es el color

In [ ]:
X_test[2000].size

In [ ]:
model.predict(np.array([X[5000]]))

In [ ]:


# PRUEBA CON COCHE AJENO AL DATASET

In [ ]:
# creo que ya estan todos importados pero por si acaso

import numpy as np
from PIL import Image
from matplotlib import pyplot as plt

In [ ]:
imgfelipe=Image.open("INPUT/fotocochesinternet/14.jpeg")
imgfelipe

In [ ]:
imgfelipe = imgfelipe.resize((64,64))

In [ ]:
imgfelipe=np.expand_dims(np.array(imgfelipe),0)

In [ ]:
imgfelipe=tf.keras.utils.normalize(imgfelipe,axis=1) 

In [ ]:
model.predict(imgfelipe)

In [ ]:
pred= model.predict(np.array(imgfelipe))

In [ ]:
c = enc.inverse_transform(pred)
prob = max(pred[0])

In [ ]:
res = c[0,0],prob
res

In [ ]:
# Buscar en doc como guardar el modelo entrenado (Guardar todo el modelo o los pesos solo) OK
# En un nuevo fichero (jupyter o .py), cargar el modelo. OK
# Crear funcion predict_color(path) => (color, prob) OK
# Crear funcion a parte que te lo ponga bonito, recibiendo el output de predict_color() OK